The increase interest for cryptocurrencies is undeniable and while many so called experts on youtube & twitter claim, with conviction, that their readings and interpretations of the markets is infallible we will attempt to generate color coded charts of crypto currencies prices with the help of Pearson correlations & MFI, money flow index.

The color of the line plos can reveal changes in trend, strength or exhaustion of an actual trend. 
Using colors allow the user looking at chart to quickly interpret how price develops with the conditions of an underlying indicator.

MFI is  a movement indicator that analyzes the buying or selling pressure of a given asset.

The last part explores how the correlation between two cryptocurrencies we choose evolves over time.

Installing required packages

In [ ]:
pip install Historic_Crypto

In [ ]:
pip install scipy

Packages & modules

In [4]:
from Historic_Crypto import HistoricalData
from scipy.stats import pearsonr
from matplotlib import cm

import matplotlib.pyplot as plt
import matplotlib.colors as col
import numpy as np
import pandas as pd 

Collecting data from Coinbase API

In [8]:
#We want to collect 1 Hour data so we set frequency = 3600 since the start 2017

frequency = 3600
from_date = '2017-01-01-00-00'

#Cryptos
cryptos_list = ['BTC-USD', 'ETH-USD']

#Querying the data 

for i in range(len(cryptos_list)):
    coin_name = cryptos_list[i]
    price_data = HistoricalData(coin_name, frequency, from_date).retrieve_data()
    price_DF = price_data[["close", "low", "high"]]
    if i == 0:
        DF = pd.DataFrame(price_DF.copy())
    else:
        DF = pd.merge(left= DF, right = price_DF, how = "left", left_index=True, right_index= True)
    
    DF.rename(columns={"close": "close - " + coin_name}, inplace = True)
    DF.rename(columns={"low": "low - " + coin_name}, inplace = True)
    DF.rename(columns={"high": "high - " + coin_name}, inplace = True)
DF.head()


Checking input parameters are in the correct format.
Formatting Dates.
Checking if user supplied is available on the CoinBase Pro API.
Connected to the CoinBase Pro API.
Ticker 'BTC-USD' found at the CoinBase Pro API, continuing to extraction.
Provisional Start: 2017-01-01T00:00:00
Provisional End: 2017-01-13T12:00:00
Data for chunk 1 of 158 extracted
Provisional Start: 2017-01-13T12:00:00
Provisional End: 2017-01-26T00:00:00
Data for chunk 2 of 158 extracted
Provisional Start: 2017-01-26T00:00:00
Provisional End: 2017-02-07T12:00:00
Data for chunk 3 of 158 extracted
Provisional Start: 2017-02-07T12:00:00
Provisional End: 2017-02-20T00:00:00
Data for chunk 4 of 158 extracted
Provisional Start: 2017-02-20T00:00:00
Provisional End: 2017-03-04T12:00:00
Data for chunk 5 of 158 extracted
Provisional Start: 2017-03-04T12:00:00
Provisional End: 2017-03-17T00:00:00
Data for chunk 6 of 158 extracted
Provisional Start: 2017-03-17T00:00:00
Provisional End: 2017-03-29T12:00:00
Data for chunk 7 of 

,close - BTC-USD,low - BTC-USD,high - BTC-USD,close - ETH-USD,low - ETH-USD,high - ETH-USD
time,,,,,,
2017-01-01 01:00:00,970.49,968.21,972.00,8.30,8.25,8.30
2017-01-01 02:00:00,967.95,967.00,971.99,8.47,8.30,8.53
2017-01-01 03:00:00,968.14,967.57,969.47,8.59,8.45,8.60
2017-01-01 04:00:00,968.10,966.64,968.62,8.53,8.49,8.60
2017-01-01 05:00:00,966.92,964.37,968.10,8.38,8.34,8.54


In [10]:
DF.to_csv('crypto_prices_Jan2017-May192022.csv')

In [9]:
DF.tail()

,close - BTC-USD,low - BTC-USD,high - BTC-USD,close - ETH-USD,low - ETH-USD,high - ETH-USD
time,,,,,,
2022-05-19 08:00:00,28928.10,28924.38,29228.32,1930.64,1929.81,1955.94
2022-05-19 09:00:00,28980.54,28813.30,29019.19,1927.32,1911.29,1936.89
2022-05-19 10:00:00,29174.42,28894.49,29238.38,1938.15,1917.62,1943.80
2022-05-19 11:00:00,29467.26,29078.87,29501.81,1962.76,1928.43,1966.62
2022-05-19 12:00:00,29460.57,29340.75,29627.82,1950.05,1944.29,1965.49
